In [1]:
import numpy as np
import json
from tempfile import TemporaryFile
import sys
from numpy import genfromtxt
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from array import *
import random
import array

## Defines FUNCTIONS

In [2]:
def act(weights, aVec, nInput, nOutput, inPattern):
  # Turn weight vector into weight matrix
  if np.ndim(weights) < 2:
      nNodes = int(np.sqrt(np.shape(weights)[0]))
      wMat = np.reshape(weights, (nNodes, nNodes))
  else:
      nNodes = np.shape(weights)[0]
      wMat = weights

  # Vectorize input
  if np.ndim(inPattern) > 1:
      nSamples = np.shape(inPattern)[0]
  else:
      nSamples = 1

  # Run input pattern through ANN    
  nodeAct  = np.zeros((nSamples,nNodes))
  nodeAct[:,0] = 1 # Bias activation
  nodeAct[:,1:nInput+1] = inPattern

  # Propagate signal through hidden to output nodes
  iNode = nInput+1
  for iNode in range(nInput+1,nNodes):
      rawAct = np.dot(nodeAct, wMat[:,iNode]).squeeze()
      nodeAct[:,iNode] = applyAct(aVec[iNode], rawAct)
      # print(nodeAct) # ADDED ADDED ADDED
  return nodeAct
#return nodeAct[:,-nOutput:]

In [3]:
def applyAct(actId, x):
  '''
  case 1  -- Linear
  case 2  -- Unsigned Step Function
  case 3  -- Sin
  case 4  -- Gausian with mean 0 and sigma 1
  case 5  -- Hyperbolic Tangent (signed)
  case 6  -- Sigmoid unsigned [1 / (1 + exp(-x))]
  case 7  -- Inverse
  case 8  -- Absolute Value
  case 9  -- Relu
  case 10 -- Cosine

  '''
  if actId == 1:   # Linear
    value = x

  if actId == 2:   # Unsigned Step Function
    value = 1.0*(x>0.0)
    #value = (np.tanh(50*x/2.0) + 1.0)/2.0

  elif actId == 3: # Sin
    value = np.sin(np.pi*x) 

  elif actId == 4: # Gaussian with mean 0 and sigma 1
    value = np.exp(-np.multiply(x, x) / 2.0)

  elif actId == 5: # Hyperbolic Tangent (signed)
    value = np.tanh(x)     

  elif actId == 6: # Sigmoid (unsigned)
    value = (np.tanh(x/2.0) + 1.0)/2.0

  elif actId == 7: # Inverse
    value = -x

  elif actId == 8: # Absolute Value
    value = abs(x)   
    
  elif actId == 9: # Relu
    value = np.maximum(0, x)   

  elif actId == 10: # Cosine
    value = np.cos(np.pi*x)
    
  else:
    value = x

  return value

In [4]:
def importNet(fileName):
  ind = np.loadtxt(fileName, delimiter=',')
  wMat = ind[:,:-1]     # Weight Matrix
  aVec = ind[:,-1]      # Activation functions

  # Create weight key (WHY IS THERE A WEIGHT KEY)
  wVec = wMat.flatten()
  wVec[np.isnan(wVec)]=0
  wKey = np.where(wVec!=0)[0] 

  return wVec, aVec, wKey

In [5]:
def show_graph_with_labels(adjacency_matrix, mylabels):
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    pos = nx.nx_agraph.graphviz_layout(gr,prog='dot')
    nx.draw(gr, node_size=200, labels=mylabels, with_labels=True)
    plt.show()

In [6]:
def get_adjacency(adjacency_matrix):
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    adj = gr.adj
    return(adj)

In [7]:
def get_degree(adjacency_matrix):
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    deg = gr.degree
    return(deg)

Order found in config.py, specs in bipedal_walker.py

1. Hull Angle (0 to 0.5)
2. Hull Angle Velocity 
3. X Velocity (-1 to 1)
4. Y Velocity (-1 to 1)
5. Hip 1 Angle (0 to 1)
6. Hip 1 Speed
7. Knee 1 Angle (0 to 2)
8. Knee 1 Speed
9. Leg 1 Contact (1 or 0)
10. Hip 2 Angle (0 to 1)
11. Hip 2 Speed
12. Knee 2 Angle (0 to 2)
13. Knee 2 Speed
14. Leg 2 Contact (1 or 0)

For Hardcore
15. Lidar 0
16. Lidar 1
17. Lidar 2
18. Lidar 3
19. Lidar 4
20. Lidar 5
21. Lidar 6
22. Lidar 7
23. Lidar 8
24. Lidar 9

In [8]:
def make_input():
    hull_angle = random.random()/2
    hull_angle_velo = (random.random()-0.5)*2
    x_velo = (random.random()-0.5)*2
    y_velo = (random.random()-0.5)*2
    hip_1_angle = random.random()-0.5
    hip_1_speed = random.random()*2
    knee_1_angle = random.random()-0.5
    knee_1_speed = random.random()*2
    leg_1_contact = random.choice([0,1])
    hip_2_angle = random.random()-0.5
    hip_2_speed = random.random()*2
    knee_2_angle = random.random()-0.5
    knee_2_speed = random.random()*2
    leg_2_contact = random.choice([0,1])
    
    input_array = np.array([hull_angle, hull_angle_velo, x_velo, y_velo, hip_1_angle,
                         hip_1_speed, knee_1_angle, knee_1_speed, leg_1_contact, hip_2_angle,
                         hip_2_speed, knee_2_angle, knee_2_speed, leg_2_contact])
    return input_array


## Import NET

In [9]:
a = importNet('log/test_best.out')
weights = a[0]
weights = weights.flatten()
activations = a[1]
activations = activations.flatten()
weightkey = a[2]
weightkey = weightkey.flatten()

#np.set_printoptions(threshold=sys.maxsize)
#print(weights)
#print(activations)
#print(weightkey)

In [10]:
weight_adjust = json.load(open('log_tool/biped.cma.4.360.best.json', 'r'))
weight_adjust = weight_adjust[0]
weight_adjust = np.array(weight_adjust)

for i in range(0,len(weightkey)):
    x = weightkey[i]
    weights[x] = weight_adjust[i]

#print(weights)

In [11]:
mydata = genfromtxt('log/test_best.out', delimiter=',')
adjacency = mydata[:,:-1]
# np.set_printoptions(threshold=sys.maxsize)
#print(type(adjacency))
#print(adjacency)

### Return the degree and neighbors of each node

In [12]:
## Neighbors
netadjacency = get_adjacency(adjacency)
netadj = pd.DataFrame(netadjacency)
netadjacency

a = []
for x in netadjacency:
    a.append(netadjacency[x])
a

#netadj

[AtlasView({27: {}, 30: {}, 36: {}, 38: {}}),
 AtlasView({1: {}, 52: {}, 106: {}}),
 AtlasView({1: {}}),
 AtlasView({1: {}, 61: {}, 90: {}}),
 AtlasView({1: {}, 62: {}, 84: {}}),
 AtlasView({31: {}, 32: {}, 35: {}, 37: {}, 40: {}, 41: {}, 43: {}}),
 AtlasView({2: {}, 57: {}, 65: {}}),
 AtlasView({2: {}, 58: {}}),
 AtlasView({2: {}, 59: {}, 60: {}, 65: {}, 78: {}, 114: {}}),
 AtlasView({2: {}, 70: {}}),
 AtlasView({2: {}, 64: {}, 65: {}, 66: {}, 97: {}, 113: {}}),
 AtlasView({2: {}, 78: {}, 107: {}}),
 AtlasView({2: {}, 68: {}}),
 AtlasView({29: {}, 33: {}, 46: {}}),
 AtlasView({4: {}, 55: {}, 56: {}, 67: {}, 74: {}, 106: {}, 110: {}, 111: {}}),
 AtlasView({4: {}, 105: {}, 107: {}, 114: {}}),
 AtlasView({4: {}, 70: {}}),
 AtlasView({50: {}}),
 AtlasView({8: {}, 84: {}}),
 AtlasView({59: {}}),
 AtlasView({9: {}, 35: {}, 77: {}, 82: {}, 96: {}, 108: {}, 111: {}, 112: {}, 113: {}}),
 AtlasView({49: {}}),
 AtlasView({11: {}, 100: {}}),
 AtlasView({28: {}}),
 AtlasView({12: {}, 53: {}, 54: {

In [13]:
## Degree
netdegree = get_degree(adjacency)
netdegree

DegreeView({1: 4, 27: 3, 30: 1, 36: 3, 38: 3, 2: 7, 31: 3, 32: 2, 35: 6, 37: 2, 40: 6, 41: 3, 43: 2, 4: 3, 29: 8, 33: 4, 46: 2, 8: 1, 50: 2, 9: 1, 59: 9, 11: 1, 49: 2, 12: 1, 28: 6, 13: 3, 42: 3, 47: 2, 58: 5, 14: 2, 39: 2, 44: 2, 17: 2, 34: 3, 48: 2, 20: 1, 26: 3, 21: 1, 25: 2, 24: 2, 45: 2, 70: 4, 82: 7, 51: 3, 111: 9, 52: 3, 106: 9, 53: 3, 54: 2, 73: 3, 103: 5, 110: 19, 55: 3, 56: 5, 67: 4, 74: 6, 57: 2, 65: 4, 105: 6, 107: 8, 114: 9, 62: 3, 101: 12, 60: 3, 78: 4, 61: 7, 90: 7, 84: 10, 63: 1, 64: 10, 66: 2, 97: 5, 113: 13, 68: 2, 69: 4, 108: 12, 71: 2, 100: 3, 72: 2, 75: 3, 76: 6, 94: 4, 77: 3, 96: 6, 112: 5, 79: 2, 80: 2, 81: 2, 83: 4, 92: 3, 99: 5, 116: 5, 85: 3, 104: 3, 86: 3, 87: 2, 88: 3, 93: 5, 98: 6, 89: 2, 91: 3, 95: 2, 102: 2, 109: 2, 115: 2})

## Test NET ACTIVITY

In [14]:
x = make_input()
print(x)

[ 0.20649133 -0.17897563  0.74435877 -0.98951711 -0.26746133  0.23165869
 -0.30492568  1.63933495  1.          0.38167291  1.69961677  0.12841535
  1.06635021  1.        ]


In [15]:
input1 = [0.2017684,-0.86527923,-0.03668345,0.05971819,-0.07059119,0.44689484,0.33507529,0.53860294,0,0.09836259,
          1.37185575,0.14996934,1.4382895,0]
activity1 = act(weights, activations, 14, 4, input1)
# print(activity1)
activityfinal1 = activity1.tolist()
activityfinal1 = activityfinal1[0]
# print(activityfinal1)

input2 = [0.08817522,-0.65542809,0.0842182,0.9432739,-0.34822493,0.45084109,-0.48459656,1.47212443,0,
          -0.27064871,0.99304459,-0.02051859,1.92493409,1]
activity2 = act(weights, activations, 14, 4, input1)
# print(activity2)
activityfinal2 = activity2.tolist()
activityfinal2 = activityfinal2[0]
# print(activityfinal2)

input3 = [0.01137174,0.41524899,0.68857528,-0.62357961,-0.32825,0.59047409,-0.16416653,0.47076078,0,
          0.08891353,0.51889859,-0.16107538,1.98948725,1]
activity3 = act(weights, activations, 14, 4, input3)
# print(activity3)
activityfinal3 = activity3.tolist()
activityfinal3 = activityfinal3[0]
# print(activityfinal3)

input4 = [0.13401449,-0.57891592,0.86738749,-0.87370349,0.08622442,1.45515756,-0.00847091,0.12981996,1,
          0.01208645,1.62544203,-0.46756413,0.08976418,1]
activity4 = act(weights, activations, 14, 4, input4)
# print(activity4)
activityfinal4 = activity4.tolist()
activityfinal4 = activityfinal4[0]
# print(activityfinal4)

input5 = [0.42944717,0.2074845,0.97696691,0.12070956,0.17939989,1.49963645,-0.49691047,1.66875721,0,
          0.03247392,0.21633115,-0.31138889,1.8906281,1]
activity5 = act(weights, activations, 14, 4, input5)
# print(activity)
activityfinal5 = activity5.tolist()
activityfinal5 = activityfinal5[0]
# print(activityfinal5)

input6 = [0.27435769,-0.95405167,-0.0687809,0.25114517,-0.28224314,0.6103709,-0.38332478,1.22043867,1,
          0.48246004,0.46512548,0.2443897,0.65039461,1]
activity6 = act(weights, activations, 14, 4, input6)
# print(activity6)
activityfinal6 = activity6.tolist()
activityfinal6 = activityfinal6[0]
# print(activityfinal6)

input7 = [0.40783188,0.06828304,-0.90034212,-0.1195795,-0.04803314,0.84565779,-0.38153606,0.54314531,0,
          -0.27854096,0.41777793,-0.47391955,0.9799453,0]
activity7 = act(weights, activations, 14, 4, input7)
# print(activity7)
activityfinal7 = activity7.tolist()
activityfinal7 = activityfinal7[0]
# print(activityfinal7)

input8 = [0.39904209,0.5447909,0.54244449,0.98677531,0.2585239,1.13785983,0.02328059,1.2579013,1,
          0.37752067,1.50983432,-0.04979908,1.31405036,1]
activity8 = act(weights, activations, 14, 4, input8)
# print(activity8)
activityfinal8 = activity8.tolist()
activityfinal8 = activityfinal8[0]
# print(activityfinal8)

input9 = [0.14200223,0.10506085,-0.88020283,0.99987185,-0.42798399,1.60094096,0.07757391,0.72301451,1,
          0.37736201,1.65605076,-0.27427311,1.14991194,0]
activity9 = act(weights, activations, 14, 4, input9)
# print(activity9)
activityfinal9 = activity9.tolist()
activityfinal9 = activityfinal9[0]
# print(activityfinal9)

input10 = [0.33446167,-0.85913756,-0.15074502,-0.09802456,0.06363981,0.88345744,-0.05966758,1.46042823,1,
           0.12733243,0.36211139,-0.28088392,1.18522888,1]
activity10 = act(weights, activations, 14, 4, input10)
# print(activity10)
activityfinal10 = activity10.tolist()
activityfinal10 = activityfinal10[0]
# print(activityfinal10)

input11 = [0.30330423,-0.54117593,-0.36597175,0.00330838,-0.41590829,0.04284109,-0.29823101,0.34049953,0,
           0.08596648,1.62302274,0.27133006,0.05819929,1]
activity11 = act(weights, activations, 14, 4, input11)
# print(activity11)
activityfinal11 = activity11.tolist()
activityfinal11 = activityfinal11[0]
# print(activityfinal11)

input12 = [0.41610037,0.87151753,-0.22808328,-0.84107837,0.41416021,1.04691405,0.27775408,1.23783319,0,
           -0.38513062,0.36859981,0.10254817,0.926992,0]
activity12 = act(weights, activations, 14, 4, input12)
# print(activity12)
activityfinal12 = activity12.tolist()
activityfinal12 = activityfinal12[0]
# print(activityfinal12)

input13 = [0.32161249,0.27427198,0.94028727,-0.30247626,0.14534802,1.78101329,-0.44978939,1.46165417,0,
           0.4254015,0.96159047,0.11461734,0.48551932,1]
activity13 = act(weights, activations, 14, 4, input13)
# print(activity13)
activityfinal13 = activity13.tolist()
activityfinal13 = activityfinal13[0]
# print(activityfinal13)

input14 = [0.16052778,-0.37562827,0.45416989,-0.32968225,0.29136561,1.86718786,-0.39983196,0.08203596,1,
           0.40871957,0.18113479,-0.21720479,0.97509864,1]
activity14 = act(weights, activations, 14, 4, input14)
# print(activity14)
activityfinal14 = activity14.tolist()
activityfinal14 = activityfinal14[0]
# print(activityfinal14)

input15 = [0.42801385,0.99356467,0.04168359,-0.03361417,0.25166841,0.87821067,0.48033997,0.07065558,0,
           -0.1544713,1.20562766,0.03173151,1.50026932,1]
activity15 = act(weights, activations, 14, 4, input15)
# print(activity15)
activityfinal15 = activity15.tolist()
activityfinal15 = activityfinal15[0]
# print(activityfinal10)

input16 = [0.28527601,0.45061613,-0.66284383,-0.0584272,-0.31829665,0.94692555,-0.27561532,1.97856582,1,
           -0.36047512,1.15321422,-0.45198038,1.2228537,0]
activity16 = act(weights, activations, 14, 4, input16)
# print(activity16)
activityfinal16 = activity16.tolist()
activityfinal16 = activityfinal16[0]
# print(activityfinal16)

input17 = [0.23329345,-0.27074835,-0.12299548,-0.61816857,-0.46219173,0.83512126,0.24580884,1.50311913,1,
           -0.22001833,0.75946016,0.01645656,1.05088389,1]
activity17 = act(weights, activations, 14, 4, input17)
# print(activity17)
activityfinal17 = activity17.tolist()
activityfinal17 = activityfinal17[0]
# print(activityfinal17)

input18 = [0.08136248,-0.62782522,-0.75699286,-0.68192183,-0.05805346,0.66516281,-0.2072248,1.00675617,1,
           -0.18761514,0.59407949,0.13985712,1.43744603,1]
activity18 = act(weights, activations, 14, 4, input18)
# print(activity18)
activityfinal18 = activity18.tolist()
activityfinal18 = activityfinal18[0]
# print(activityfinal18)

input19 = [0.48040456,0.83798572,-0.25178012,-0.1470078,0.02746377,0.60112859,0.00473028,0.43452105,0,
           0.39356353,0.75204053,-0.30779139,1.08353653,1]
activity19 = act(weights, activations, 14, 4, input19)
# print(activity19)
activityfinal19 = activity19.tolist()
activityfinal19 = activityfinal19[0]
# print(activityfinal19)

input20 = [0.32653235,-0.93542982,0.17994684,-0.21982811,-0.2092894,1.28239571,-0.4408143,1.34492513,0,
           -0.46394338,0.97576477,0.36840567,0.97906551,0]
activity20 = act(weights, activations, 14, 4, input20)
# print(activity20)
activityfinal20 = activity20.tolist()
activityfinal20 = activityfinal20[0]
# print(activityfinal20)

## EXPORT DATA

In [16]:
## export activity as .csv, everything else as .json as it isn't exactly tabular 

finalactivity = pd.DataFrame([activations,
                              activityfinal1,
                           activityfinal2,
                           activityfinal3,
                           activityfinal4,
                           activityfinal5,
                           activityfinal6,
                           activityfinal7,
                           activityfinal8,
                           activityfinal9,
                           activityfinal10,
                              activityfinal11,
                              activityfinal12,
                              activityfinal13,
                              activityfinal14,
                              activityfinal15,
                              activityfinal16,
                              activityfinal17,
                              activityfinal18,
                              activityfinal19,
                              activityfinal20
                             ])

finalactivity.to_csv('activity_2-26-21.csv', index=False)

finalinfo = pd.DataFrame([netdegree,
                         netadjacency,
                         a])

finalinfo.to_csv('info_2-26-21.csv', index=False)

adjacency = adjacency.tolist()

with open("adjacency_2-26-21.json", "w") as f:
    json.dump(adjacency, f)